In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:


# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Define data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
train_datagen.fit(train_images)


In [3]:

# Load pre-trained VGG16 model without including top dense layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze all layers in base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom dense layers for CIFAR-10 classification
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer=optimizers.Adam(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [4]:

# Train the model
history = model.fit(train_datagen.flow(train_images, train_labels, batch_size=64),
                    steps_per_epoch=len(train_images) / 64, epochs=10, 
                    validation_data=(test_images, test_labels))


Epoch 1/10


781/781 [==============================] - 169s 214ms/step - loss: 1.6103 - accuracy: 0.1023 - val_loss: 1.3481 - val_accuracy: 0.1471
Epoch 2/10
781/781 [==============================] - 157s 200ms/step - loss: 1.4107 - accuracy: 0.0983 - val_loss: 1.2726 - val_accuracy: 0.1143
Epoch 3/10
781/781 [==============================] - 156s 200ms/step - loss: 1.3647 - accuracy: 0.0983 - val_loss: 1.2370 - val_accuracy: 0.1087
Epoch 4/10
781/781 [==============================] - 161s 206ms/step - loss: 1.3412 - accuracy: 0.0983 - val_loss: 1.2188 - val_accuracy: 0.1087
Epoch 5/10
781/781 [==============================] - 167s 214ms/step - loss: 1.3161 - accuracy: 0.0992 - val_loss: 1.2010 - val_accuracy: 0.1120
Epoch 6/10
781/781 [==============================] - 166s 212ms/step - loss: 1.2958 - accuracy: 0.0974 - val_loss: 1.1875 - val_accuracy: 0.1137
Epoch 7/10
781/781 [==============================] - 163s 208ms/step - loss: 1.2898 - accuracy: 0.0964 - val_loss: 1.1735

In [5]:

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 31s 98ms/step - loss: 1.1601 - accuracy: 0.1207
Test accuracy: 0.12070000171661377
